<a href="https://colab.research.google.com/github/juankchb/Reconocimiento_facial/blob/main/Codigos/capturar_Rostro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import cv2
import os
import imutils
import glob
import numpy as np

In [2]:
!git clone https://github.com/juankchb/Reconocimiento_facial.git

Cloning into 'Reconocimiento_facial'...
remote: Enumerating objects: 88, done.
remote: Counting objects: 100% (88/88), done.
remote: Compressing objects: 100% (73/73), done.
remote: Total 88 (delta 8), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (88/88), 2.83 MiB | 6.42 MiB/s, done.
Resolving deltas: 100% (8/8), done.


In [3]:
# Directorio de datos
dataPath = '/content/Reconocimiento_facial/Imagenes'

# Obtiene la lista de nombres de persona a partir de las carpetas en dataPath
personNames = [name for name in os.listdir(dataPath) if os.path.isdir(os.path.join(dataPath, name))]

# Directorio que contiene los videos
videos_dir = '/content/Reconocimiento_facial/Videos_deteccion_rostro/'

faceClassif = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Bucle for para procesar cada persona
for personName in personNames:
    # Ruta completa a la carpeta de la persona
    personPath = os.path.join(dataPath, personName)

    # Comprueba si la carpeta de la persona existe, si no, la crea
    if not os.path.exists(personPath):
        print('Carpeta creada:', personPath)
        os.makedirs(personPath)

    # Nombre del video correspondiente a la persona
    video_name = f'{personName}.mp4'
    video_path = os.path.join(videos_dir, video_name)

    # Verifica si el video existe antes de procesarlo
    if os.path.exists(video_path):
        # Abre el archivo de video
        cap = cv2.VideoCapture(video_path)

        # Inicializa el contador para las imágenes capturadas
        count = 0

        while True:
            ret, frame = cap.read()
            if not ret:
                break

            frame = imutils.resize(frame, width=640)
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            auxFrame = frame.copy()
            faces = faceClassif.detectMultiScale(gray, 1.3, 5)

            for (x, y, w, h) in faces:
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
                rostro = auxFrame[y:y + h, x:x + w]
                # Redimensiona la imagen del rostro a 150x150 píxeles
                rostro = cv2.resize(rostro, (150, 150), interpolation=cv2.INTER_CUBIC)
                # Guarda la imagen en escala de grises
                cv2.imwrite(os.path.join(personPath, f'{personName}_rostro_{count}.jpg'), rostro, [cv2.IMWRITE_JPEG_QUALITY, 100])
                count += 1

            k = cv2.waitKey(1)
            if k == 27 or count >= 301:
                break

        cap.release()

cv2.destroyAllWindows()

#Entrenar_Red_N

In [10]:
# Directorio de datos
dataPath = '/content/Reconocimiento_facial/Imagenes'

# Obtiene la lista de nombres de persona a partir de las carpetas en dataPath
personNames = [name for name in os.listdir(dataPath) if os.path.isdir(os.path.join(dataPath, name))]

# Directorio que contiene los videos
videos_dir = '/content/Reconocimiento_facial/Videos_deteccion_rostro/'

# Utiliza glob para obtener la lista de archivos de video en el directorio
video_files = glob.glob(os.path.join(videos_dir, '*.mp4'))

faceClassif = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Listas para almacenar las imágenes de rostros y las etiquetas (nombres)
facesData = []
labels = []

# Bucle for para procesar cada persona
for personName in personNames:
    # Ruta completa a la carpeta de la persona
    personPath = os.path.join(dataPath, personName)

    # Nombre del video correspondiente a la persona
    video_name = f'{personName}.mp4'
    video_path = os.path.join(videos_dir, video_name)

    # Verifica si el video existe antes de procesarlo
    if os.path.exists(video_path):
        # Abre el archivo de video
        cap = cv2.VideoCapture(video_path)

        while True:
            ret, frame = cap.read()
            if not ret:
                break

            frame = imutils.resize(frame, width=640)
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  # Convierte a escala de grises
            auxFrame = frame.copy()
            faces = faceClassif.detectMultiScale(gray, 1.3, 5)

            for (x, y, w, h) in faces:
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
                rostro = auxFrame[y:y + h, x:x + w]
                # Redimensiona la imagen del rostro a 150x150 píxeles
                rostro = cv2.resize(rostro, (150, 150), interpolation=cv2.INTER_CUBIC)
                # Convierte la imagen a escala de grises
                rostro = cv2.cvtColor(rostro, cv2.COLOR_BGR2GRAY)
                # Agrega la imagen redimensionada a la lista
                facesData.append(rostro)
                # Usa el nombre de la persona como etiqueta
                labels.append(personName)

        cap.release()

# Convertir los nombres de persona a números únicos
label_ids = {name: idx for idx, name in enumerate(set(labels))}
labels = [label_ids[name] for name in labels]
labels = np.array(labels, dtype=np.int32)

# Entrenar el reconocedor de rostros
face_recognizer = cv2.face_LBPHFaceRecognizer.create()
print("Entrenando el reconocedor de rostros...")
face_recognizer.train(facesData, labels)

# Almacenar el modelo entrenado
model_path = '/content/modeloLBPHFace.xml'
face_recognizer.save(model_path)
print(f"Modelo almacenado en {model_path}")

Entrenando el reconocedor de rostros...
Modelo almacenado en /content/modeloLBPHFace.xml


#Reconocimiento Facial